In [1]:
import numpy as np
# from cdlib import algorithms, viz, evaluation
import networkx as nx
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy

In [2]:
with open('../pkl/global_weekly_tensor2.pkl', 'rb') as f:
    travel_tensors = pickle.load(f) 
    
# with open('../pkl/global_serial_graphs.pkl', 'rb') as f:
#     serial_graphs = pickle.load(f) 

In [3]:
travel_tensors.shape

(200, 226, 452)

In [4]:
np.sum(travel_tensors>0) / (200*226*226*2)

0.003931004777194769

In [5]:
aj = travel_tensors[16]
np.sum(aj>0)

204

In [6]:
df = pd.read_csv('../csv/global_travel_df.csv')
df = df.iloc[:,1:]

In [7]:
df['First_checkin'] = pd.to_datetime(df['First_checkin'])
df['Last_checkin'] = pd.to_datetime(df['Last_checkin'])

In [8]:
home_df = df.groupby('trip_id').head(1)
home_df.rename(columns={'node':'home'}, inplace=True)

/Users/gimpuleumoe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
home_df

,trip_number,trip_id,home,First_checkin,Last_checkin
0,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12
1,1,7dee487f12,US,2018-07-11 21:09:27,2018-07-12 00:23:41
4,2,512a66d10e,EC,2017-07-08 15:32:39,2017-07-09 18:05:26
6,3,d3c75249d3,US,2019-04-18 16:57:58,2019-04-20 16:40:10
9,4,8da79163fc,DE,2017-06-24 21:38:12,2017-07-01 09:43:10
...,...,...,...,...,...
1364012,29994,10ca9a2259,IT,2016-09-07 08:17:50,2016-09-08 19:08:28
1364016,29995,cd302411cf,PH,2018-06-09 23:39:00,2018-06-13 09:52:59
1364018,29996,8d07c043e6,US,2017-03-03 17:21:22,2017-03-04 14:57:41
1364060,29999,4c2d0c689e,US,2018-02-07 22:13:58,2018-02-08 00:55:40


In [10]:
merged_df = pd.merge(df, home_df[['trip_id','home']], how='inner',left_on=['trip_id'],right_on=['trip_id'])

In [11]:
merged_df 

,trip_number,trip_id,node,First_checkin,Last_checkin,home
0,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12,US
1,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12,US
2,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12,US
3,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12,US
4,1,d8799616e3,US,2018-10-03 07:18:32,2018-10-06 05:33:12,US
...,...,...,...,...,...,...
1819576,30000,a124677fc3,MX,2018-12-10 16:39:44,2018-12-12 20:14:24,MX
1819577,30000,a124677fc3,MX,2018-12-10 16:39:44,2018-12-10 17:20:27,MX
1819578,30000,a124677fc3,MX,2018-12-11 13:41:43,2018-12-11 13:41:43,MX
1819579,30000,a124677fc3,MX,2018-12-11 16:26:33,2018-12-11 16:38:33,MX


In [12]:
travel_df = pd.read_csv('../csv/global_travel_df.csv')
countries = travel_df['node'].unique().T
label_dict = dict(zip(countries, range(226)))

In [13]:
country_num = len(countries)
week_slices = pd.date_range('2016-01-04', periods=200+1, freq='W')

home_adj_list = []

for i in range(95):
    print(i)
    home_adj = np.zeros([country_num,country_num])
    start = week_slices[104 +i]
    end = week_slices[105 +i]
    df_intime = merged_df[(merged_df['First_checkin']>=start) & (merged_df['First_checkin'] < end)]
    df_intime = df_intime[(df_intime['node']!=df_intime['home'])]
    
    for index, row in df_intime.iterrows():
        r = label_dict[row['home']]
        c = label_dict[row['node']]
        if(r!=c):
            home_adj[r,c] +=1
    home_adj_list.append(home_adj)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [14]:
week_slices[156-52]

Timestamp('2018-01-07 00:00:00', freq='W-SUN')

In [15]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

save_object(home_adj_list, '../pkl/[weekly]home_adj_from20180107_2.pkl')

In [16]:
np.array(home_adj_list).shape

(95, 226, 226)